In [1]:
import os

import numpy as np
from beartype.typing import Sequence


def _load(path) -> dict[str, int | float]:
    try:
        indices = np.load(os.path.join(path, "_fusion", "indices.npz"))
        trajectory = np.load(os.path.join(path, "_lidar", "pose.npz"))
        vel = np.diff(trajectory["pos"], axis=0)
        return {
            "n": indices['indices'].shape[0],
            "t": (trajectory['t'][-1] - trajectory['t'][0]) / 60 / 60,
            "d": np.sum(np.linalg.norm(vel, axis=1)) / 1000
        }
    except Exception as e:
        print(path, e)
        return {"n": 0, "t": 0.0, "d": 0.0}


def _total(data: Sequence[dict[str, int | float]]) -> dict[str, int | float]:
    return {k: sum(x[k] for x in data) for k in data[0]}


def report(traces: dict[str, dict[str, int | float]]):
    total = _total(list(traces.values()))

    # out = [
    #     r"\begin{array}{l r r r}",
    #     f"\\text{{total}} & {total['n']} & {total['t']:.02f}h & "
    #     f"{total['d']:.02f}km" + r" \\",
    #     r"\hline"]
    # for n, s in traces.items():
    #     out.append(
    #         f"\\texttt{{{n}}} & {s['n']} & {s['t']:.02f}h & {s['d']:.02f}km"
    #         + r" \\")
    # out.append(r"\end{array}")

    # return Latex("\n".join(out))
    # print('\n'.join(out))

    print(f"{'total':18}{total['n']:8}{total['t']:8.02f}h{total['d']:8.02f}km")
    print("-" * (18 + 8 + 8 + 8 + len('hkm')))
    for n, s in traces.items():
        print(f"{n:18}{s['n']:8}{s['t']:8.02f}h{s['d']:8.02f}km")

    return total


def load1(path):
    return {
        trace: _load(os.path.join(path, trace))
        for trace in sorted(os.listdir(path))}


def load2(path):
    return {
        building: _total([
            _load(os.path.join(path, building, dataset))
            for dataset in sorted(os.listdir(os.path.join(path, building)))])
        for building in sorted(os.listdir(path))}

In [3]:
splits = {
    "indoor": load2("/data/radarhd2/indoor"),
    "outdoor": load1("/data/radarhd2/outdoor"),
    "bike": load1("/data/radarhd2/bike")
}
totals = {k: _total(list(v.values())) for k, v in splits.items()}
grand_total = {k: sum(v[k] for v in totals.values()) for k in 'ntd'}

print("")
print("-" * 45)
print(f"{'grand total':18}{grand_total['n']:8}{grand_total['t']:8.02f}h{grand_total['d']:8.02f}km")
print("-" * 45)
print("")

for k, v in splits.items():
    report(v)
    print("\n")


---------------------------------------------
grand total        1014417   28.91h  266.71km
---------------------------------------------

total               309726    8.92h   31.94km
---------------------------------------------
baker                16040    0.46h    1.62km
cfa                  13589    0.39h    1.32km
cic                  18540    0.53h    1.73km
doherty              15205    0.44h    1.64km
gates                28519    0.82h    2.95km
hamburg              21350    0.62h    2.18km
hunt                 11747    0.34h    1.14km
morrison              7765    0.23h    0.79km
nsh                  24675    0.71h    2.42km
porter               15119    0.43h    1.51km
posner               21658    0.61h    2.05km
roberts              11077    0.32h    1.21km
scaife               10041    0.29h    1.00km
scott                14704    0.42h    1.60km
tepper               30331    0.87h    3.31km
uc                   26968    0.79h    3.01km
wean                 22398    0.